<a href="https://colab.research.google.com/github/Dyakonov/Colab/blob/main/VK_dj_simpleLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Делаем RNN на эмбедингах
* на основе https://github.com/nikhilbarhate99/Char-RNN-PyTorch/blob/master/CharRNN.py

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils.rnn as rnn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import numpy as np
import time
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np

# import shakespeare_data as sh

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cuda'

## загрузить данные

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
import pickle
with open("/content/drive/My Drive/VK2/lst1.pkl", "rb") as fp1:
  data1 = pickle.load(fp1)

with open("/content/drive/My Drive/VK2/lst2.pkl", "rb") as fp2:
  data2 = pickle.load(fp2)

len(data1), len(data2)

(1000183, 1000183)

In [ ]:
n_data = len(data1)

In [ ]:
data1[0]

array([ 35236, 186864,  58724, 155390, 153029,  28510,  62046, 184658,
        58738, 154938,   6009, 224625,   3814, 201408, 172560, 121582,
        77054,  82504, 161508, 141730, 134408, 182023,  55382, 158027,
       178379, 215675, 221030,  22156,  51798, 167951, 219579,  54416,
       198166], dtype=int32)

In [ ]:
data2[0]

array([224625, 224625, 224625, 224625, 224625, 224625, 224625, 224625,
       224625, 224625, 224625, 201408, 201408, 219579, 219579, 219579,
       219579, 219579, 219579, 219579, 219579, 219579, 219579, 219579,
       219579, 219579, 219579, 219579, 219579, 219579,  54416, 198166,
       167423], dtype=int32)

In [ ]:
import numpy as np
Wemb = np.load('/content/drive/My Drive/VK2/XX.mtx.npy')
Wemb = torch.tensor(Wemb)
Wemb.shape

torch.Size([227606, 177])

In [ ]:
#from scipy.sparse.linalg import svds
# Wemb, _, _ = svds(Wemb, k=160, return_singular_vectors='u')

In [ ]:
# данные
#data = [[1, 9, 8, 7, 4], [1, 2, 3], [0, 2, 4, 6, 8], [9, 8, 7, 6]]

# матрица представлений
#Wemb = torch.tensor(np.random.rand(10, 4))

In [ ]:
Wemb

tensor([[-0.1674,  0.0490,  0.0555,  ...,  0.0404, -0.0410,  0.1514],
        [-0.1314,  0.0330,  0.0598,  ..., -0.1735, -0.2696,  0.2193],
        [ 0.0422, -0.0618, -0.0458,  ...,  0.2930, -0.0840,  0.1581],
        ...,
        [-0.0499,  0.0205,  0.0835,  ...,  0.0903, -0.0424,  0.1637],
        [-0.0417, -0.0633, -0.0019,  ..., -0.1141, -0.0324,  0.0979],
        [ 0.0119,  0.0842, -0.0054,  ...,  0.0210, -0.1287,  0.1128]])

## Модель

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RNN(nn.Module):
    def __init__(self, dict_size, emb_size, output_size, hidden_size, num_layers, W):
        super(RNN, self).__init__()
        self.embedding = nn.Embedding(dict_size, emb_size)
        self.rnn = nn.LSTM(input_size=emb_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)

        if W is not None:
          self.embedding.weight.requires_grad = False # так не работает_(False)
          self.decoder.weight.requires_grad = False # так не работает_(False)
          self.embedding.weight.copy_(W) # сначала убрали градиенты
          self.decoder.weight.copy_(W) # .T



    def forward(self, input_seq, hidden_state):
        embedding = self.embedding(input_seq)
        output, hidden_state = self.rnn(embedding, hidden_state)
        output = self.decoder(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())

## Параметры

In [ ]:
hidden_size = 177 #####312     # size of hidden state
# seq_len = 100       # length of LSTM sequence
num_layers = 3      # num of layers in LSTM layer stack
lr = 0.0001          # learning rate
epochs = 1        # max number of epochs
op_seq_len = 227606    # total num of characters in output test sequence
vocab_size = 227606
#load_chk = False    # load weights from save_path directory to continue training

save_path = '/content/drive/My Drive/VK2/savernn.pth' # to save

In [ ]:
# model instance
rnn = RNN(dict_size=vocab_size, emb_size=hidden_size, output_size=vocab_size, hidden_size=hidden_size, num_layers=num_layers, W=Wemb).to(device) #


# loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(rnn.parameters(), lr=lr)

In [ ]:
from time import time
from IPython.display import clear_output

In [ ]:
rnn.load_state_dict(torch.load('/content/drive/My Drive/VK2/savernn.pth'))

<All keys matched successfully>

In [ ]:
#save_path = '/content/drive/My Drive/VK2/savernn.pth'
#torch.save(rnn.state_dict(), save_path)

In [ ]:
np.random.permutation(15)

array([ 5, 14, 13,  4,  7,  3,  6,  9, 10,  8,  2, 12, 11,  1,  0])

## Обучение

In [ ]:
# training loop
for i_epoch in range(1, epochs+1):

    n = 0
    tm = time()
    running_loss = 0
    hidden_state = None


    for i in np.random.permutation(n_data):
      # data tensor on device
      input_seq = torch.tensor(data1[i]).to(device)
      input_seq = torch.unsqueeze(input_seq, dim=1)

      target_seq = torch.tensor(data2[i]).type(torch.LongTensor).to(device)
      target_seq = torch.unsqueeze(target_seq, dim=1)

      hidden_state = None

      # forward pass
      output, hidden_state = rnn(input_seq, hidden_state)

      # compute loss
      loss = loss_fn(torch.squeeze(output), torch.squeeze(target_seq))
      running_loss += loss.item()
      n+=1

      # compute gradients and take optimizer step
      optimizer.zero_grad()
      loss.backward()
      nn.utils.clip_grad_norm_(rnn.parameters(), 1.0) # НЕ ПРОБОВАЛ!!!
      optimizer.step()

      if (n % 1000) == 0:
        clear_output(wait=True)
        print(f'iteration={n}, time={(time() - tm)/n:.5f}, tek_loss={loss.item():.5f}, avg_loss={running_loss/n:.5f}')

      if (n % 100000) == 0:
        torch.save(rnn.state_dict(), save_path)

    # print loss and save weights after every epoch
    print("Epoch: {0} \t Loss: {1:.8f}".format(i_epoch, running_loss/n))
    torch.save(rnn.state_dict(), save_path)





# data    iteration=586300, time=0.02145, tek_loss=10.58368, avg_loss=11.33329

iteration=694000, time=0.02497, tek_loss=13.15814, avg_loss=9.54445


KeyboardInterrupt: ignored

In [ ]:
import pickle
with open("/content/drive/My Drive/VK2/lst.pkl", "rb") as fp:
  data = pickle.load(fp)

In [ ]:
users = np.load('/content/drive/My Drive/VK2/users.mtx.npy')
users.shape

(200000,)

In [ ]:
A = np.zeros((len(users), 177))

In [ ]:
rnn.decoder = nn.Identity(177)

In [ ]:
tm = time()

for t, user in enumerate(users):
      # data tensor on device
      input_seq = torch.tensor(data[user]).to(device)
      input_seq = torch.unsqueeze(input_seq, dim=1)

      hidden_state = None

      # forward pass
      output, hidden_state = rnn(input_seq, hidden_state)

      A[t,:] = output.squeeze()[-1,:].detach().cpu().numpy()

      if (t % 1000) == 0:
        clear_output(wait=True)
        print(f'iteration={t}, time={(time() - tm)/(t+1):.5f}')

iteration=199000, time=0.00534


In [ ]:
A

array([[ 2.44980708e-01,  2.83159548e-04,  9.16144159e-03, ...,
        -3.69317949e-01, -9.82678950e-01, -9.99981761e-01],
       [ 9.99818683e-01, -8.05878919e-03,  4.94417213e-02, ...,
        -1.00000000e+00,  9.99999583e-01, -9.99994874e-01],
       [ 7.41303444e-01, -6.08208239e-01,  3.87117088e-01, ...,
        -5.65984666e-01,  9.99999762e-01, -4.64842021e-01],
       ...,
       [ 8.27734530e-01,  7.24312514e-02,  4.99795228e-02, ...,
        -9.89626467e-01, -5.11101007e-01, -9.99991655e-01],
       [ 1.44920737e-01, -7.57850409e-01,  9.99975085e-01, ...,
        -4.48987752e-01,  1.00000000e+00, -9.99952793e-01],
       [ 9.95108068e-01, -4.74895169e-05,  7.44295239e-01, ...,
        -9.79716837e-01, -5.51866472e-01, -9.99453485e-01]])

In [ ]:
np.save('/content/drive/My Drive/VK2/A.mtx', A)

In [ ]:
del data

In [ ]:
output.device, target_seq.device

(device(type='cuda', index=0), device(type='cpu'))

In [ ]:
torch.squeeze(output)

tensor([[-0.0901,  0.0391,  0.0022,  ...,  0.0097,  0.0289,  0.0580],
        [-0.1205,  0.0350, -0.0057,  ..., -0.0034,  0.0421,  0.0627],
        [-0.1376,  0.0321, -0.0079,  ..., -0.0091,  0.0496,  0.0649],
        ...,
        [-0.1611,  0.0320, -0.0074,  ..., -0.0051,  0.0627,  0.0648],
        [-0.1608,  0.0320, -0.0076,  ..., -0.0053,  0.0629,  0.0642],
        [-0.1606,  0.0319, -0.0079,  ..., -0.0060,  0.0632,  0.0643]],
       device='cuda:0', grad_fn=<SqueezeBackward0>)

In [ ]:
torch.squeeze(target_seq, dtype=Long)

NameError: ignored

In [ ]:
len(data)

NameError: ignored

In [ ]:
    # sample / generate a text sequence after every epoch
    hidden_state = None

 # random character from data to begin
    x = torch.tensor([1, 3]).to(device)
    x = torch.unsqueeze(x, dim=1)
    input_seq = x

    print("----------------------------------------")
    for t in range(10):
        # forward pass
        output, hidden_state = rnn(input_seq, hidden_state)

        ##### print(output, end='')

        # construct categorical distribution and sample a character
        # output = F.softmax(torch.squeeze(output), dim=0)
        # dist = Categorical(output)
        #index = dist.sample()

        # print the sampled character
        # print(index, end='')

    print("\n----------------------------------------")

NameError: ignored

In [ ]:
iteration=1100, time=0.03170, tek_loss=11.52488, avg_loss=11.81203
iteration=1100, time=0.09800, tek_loss=11.17373, avg_loss=11.49727 без инициализации
iteration=1300, time=0.02554, tek_loss=11.56723, avg_loss=11.45241 1 слой
#iteration=2900, time=0.02170, tek_loss=11.97592, avg_loss=11.65800 уменьшил вложение до 160
# rnn

RNN(
  (embedding): Embedding(227606, 312)
  (rnn): LSTM(312, 312, num_layers=3)
  (decoder): Linear(in_features=312, out_features=227606, bias=True)
)

In [ ]:
i_epoch

NameError: ignored

In [ ]:
for param in rnn.parameters():
    print(param[:3])

tensor([[ 1.0458e-01,  4.7880e-02,  3.0944e-02, -3.5116e-02, -2.6452e-02,
         -1.8017e-02,  1.6048e-02, -4.5419e-02,  3.1146e-04,  9.3888e-02,
          6.8062e-02, -4.1069e-02,  6.2561e-02,  1.8975e-03, -6.3457e-02,
         -5.6746e-03,  1.5704e-02, -6.1965e-02,  1.4970e-02, -2.4888e-02,
          5.6331e-02,  1.4982e-02,  4.7225e-02,  2.1182e-02,  7.9994e-02,
         -2.1951e-02, -5.7099e-02,  1.6034e-02, -3.0449e-02,  2.4281e-02,
          1.0529e-02, -2.3831e-02,  1.1418e-02, -6.7547e-02, -2.4365e-02,
          5.2528e-02, -2.0069e-02,  4.9014e-03, -3.6743e-02,  3.1437e-03,
         -1.8055e-02,  2.4830e-01,  7.3590e-02,  1.4884e-02, -4.2709e-02,
          4.1417e-03, -9.0915e-03, -6.6086e-03,  1.5600e-02, -7.8404e-03,
          4.3871e-02, -4.1044e-02,  2.0118e-02, -5.2628e-02,  7.7389e-02,
         -1.1714e-03,  1.8893e-02, -4.1031e-02,  3.1918e-02, -3.8071e-02,
          3.1811e-03,  8.0917e-02, -3.7467e-02,  2.0274e-02, -2.4794e-02,
          1.4012e-01,  8.5882e-02, -3.

In [ ]:
Wemb

tensor([[0.5704, 0.4398, 0.5965, 0.1980],
        [0.1603, 0.8624, 0.7394, 0.8273],
        [0.6408, 0.4037, 0.8793, 0.6711],
        [0.7586, 0.5244, 0.6401, 0.6179],
        [0.0460, 0.5391, 0.8949, 0.9827],
        [0.4686, 0.4335, 0.2629, 0.9856],
        [0.8061, 0.4220, 0.8255, 0.2514],
        [0.0526, 0.1697, 0.2963, 0.2130],
        [0.7402, 0.8136, 0.3871, 0.2879],
        [0.2652, 0.0402, 0.6527, 0.3249]], dtype=torch.float64)

In [ ]:
for param in rnn.parameters():
    print(param[:3])

tensor([[0.5704, 0.4398, 0.5965, 0.1980],
        [0.1603, 0.8624, 0.7394, 0.8273],
        [0.6408, 0.4037, 0.8793, 0.6711]])
tensor([[ 0.4362,  0.1147,  0.0451, -0.1344],
        [ 0.5495,  0.4788,  0.0143,  0.4321],
        [-0.2826,  0.3624,  0.4951,  0.4399]], grad_fn=<SliceBackward0>)
tensor([[-0.1491, -0.1508, -0.0161, -0.4091],
        [-0.5806,  0.4944, -0.1787, -0.5823],
        [ 0.3364,  0.0742, -0.3260, -0.4880]], grad_fn=<SliceBackward0>)
tensor([-0.2303,  0.3446, -0.0392], grad_fn=<SliceBackward0>)
tensor([ 0.4675,  0.4696, -0.2197], grad_fn=<SliceBackward0>)
tensor([[ 0.0812,  0.0496, -0.2597,  0.2574],
        [-0.2986,  0.2188, -0.4719, -0.3133],
        [ 0.0777,  0.4203, -0.4143,  0.2328]], grad_fn=<SliceBackward0>)
tensor([[-0.3277,  0.0181,  0.1832, -0.1319],
        [ 0.2915,  0.3378,  0.1421, -0.1265],
        [-0.2072, -0.3991, -0.4702, -0.3347]], grad_fn=<SliceBackward0>)
tensor([-0.0266,  0.0412, -0.3356], grad_fn=<SliceBackward0>)
tensor([-0.2209,  0.3401, -

In [ ]:
rnn.embedding.weight.requires_grad
#data.copy_(Wemb)

True

In [ ]:
tmp = nn.Embedding(2, 3)

In [ ]:
tmp.weight.requires_grad_(False)

Parameter containing:
tensor([[ 0.6603, -0.3301, -1.4681],
        [ 0.7815,  0.5942,  1.5088]])

In [ ]:
tmp.weight

Parameter containing:
tensor([[ 0.6603, -0.3301, -1.4681],
        [ 0.7815,  0.5942,  1.5088]])

## разбираемся с размерностями параметров

In [ ]:
data_ptr = 1000
input_seq = data[data_ptr : data_ptr+seq_len]
target_seq = data[data_ptr+1 : data_ptr+seq_len+1]
print (input_seq.shape) # окно x 1
input_seq.T

torch.Size([100, 1])


tensor([[ 85,  81,   0,  96,  92,  77,  81,  84,  87,  90, 101,   6,   0,  74,
          92,  77,  91,  87,   0,  75,   0,  90,  91,  78,  88,  81,   0,  75,
          90,  78,  76,  87,   0,  77,  75,  73,   0,  96,  78,  84,  87,  75,
          78,  83,  73,  20,   0,  90,  73,  85,  73,   0,  66,  73,  76,  73,
           0,  81,   0,  80,  73,  94,  87,  77, 104,  98,  73, 104,  90, 104,
           0,  75,   0,  83,  89,  81,  83,  78,   0,  56,  73,  91, 101,   8,
           0,   7,   0,  48,  73,   0,  88,  87,  77,  83,  89,  73,  77,  78,
          91,  90]], device='cuda:0')

In [ ]:
output, hidden_state = rnn(input_seq, None)
print (output.shape, # окно x 1 x число символов
       hidden_state[0].shape, # число слоёв x 1 x размерность состояния
       hidden_state[1].shape) # число слоёв x 1 x размерность состояния

(torch.Size([100, 1, 105]), torch.Size([3, 1, 512]), torch.Size([3, 1, 512]))

In [ ]:
print(torch.squeeze(output).shape, # убираем фиктивную размерность
      torch.squeeze(target_seq).shape)

torch.Size([100, 105]) torch.Size([100])


In [ ]:
embedding = rnn.embedding(input_seq)
print (embedding.shape) # окно x 1 x число символов

torch.Size([100, 1, 105])


In [ ]:
output, hidden_state = rnn.rnn(embedding, hidden_state)
print (output.shape, # окно x 1 x размерность состояния
       hidden_state[0].shape,  # число слоёв x 1 x размерность состояния
       hidden_state[1].shape) # число слоёв x 1 x число символов

torch.Size([100, 1, 512]) torch.Size([3, 1, 512]) torch.Size([3, 1, 512])


In [ ]:
output = rnn.decoder(output)
print (output.shape)  # окно x 1 x число символов

torch.Size([100, 1, 105])
